In [1]:
import tensorflow as tf
import torch
import numpy as np

2024-02-09 10:14:12.722666: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-09 10:14:13.519172: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-09 10:14:13.521877: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-09 10:14:15.167327: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [22]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.20f}".format(x)})
torch.set_printoptions(precision=20)
# x = tf.constant([2.5], dtype=tf.bfloat16)
x = tf.random.uniform([3,4], dtype=tf.bfloat16)
x1 = torch.tensor(tf.cast(x, tf.float32).numpy())
x1 = x1.to(dtype=torch.bfloat16)
print(x)
print(x1)

tf.Tensor(
[[0.335938 0.992188 0.742188 0.960938]
 [0.507812 0.546875 0.804688 0.742188]
 [0.476562 0.78125 0.53125 0.320312]], shape=(3, 4), dtype=bfloat16)
tensor([[0.33593750000000000000, 0.99218750000000000000, 0.74218750000000000000,
         0.96093750000000000000],
        [0.50781250000000000000, 0.54687500000000000000, 0.80468750000000000000,
         0.74218750000000000000],
        [0.47656250000000000000, 0.78125000000000000000, 0.53125000000000000000,
         0.32031250000000000000]], dtype=torch.bfloat16)


In [ ]:
# class LayerNorm(tf.Module):
#     def __init__(self, weights, biases, eps=1e-12, name=None):
#         super().__init__(name)
#         # w's and b's are both 1d (768)
#         self.weights = weights
#         self.biases = biases
#         self.eps = eps
#     # @tf.function
#     def __call__(self, input):
#         self.weights = tf.cast(self.weights, dtype=tf.float32)
#         self.biases = tf.cast(self.biases, dtype=tf.float32)
#         # input is 2d (batch of 1)
#         mean = tf.math.reduce_mean(input, axis=-1, keepdims=True) # (512, 1) i think
#         std = tf.math.reduce_std(input, axis=-1, keepdims=True)
#         mean = tf.cast(mean, dtype=tf.float32)
#         std = tf.cast(std, dtype=tf.float32)
#         input = tf.cast(input, dtype=tf.float32)
#         return tf.cast(self.weights * (input - mean) / (std + self.eps) + self.biases, dtype=tf.float16)

In [155]:
class LayerNorm(tf.Module):
    def __init__(self, weights, biases, eps=1e-12, name=None):
        super().__init__(name)
        # w's and b's are both 1d (768)
        self.weights = weights
        self.biases = biases
        self.eps = eps
    # @tf.function
    def __call__(self, input):
        # eps = tf.constant(self.eps, dtype=tf.float16)
        # input is 2d (batch of 1)
        mean = tf.math.reduce_mean(input, axis=-1, keepdims=True) # (512, 1) i think
        std = tf.math.reduce_std(input, axis=-1, keepdims=True)
        x = std + self.eps
        # x = 1 / x # is there another way to do this?
        x = tf.math.reciprocal(x)
        return self.weights * (input - mean) * x + self.biases

        # return self.weights * (input - mean) / (std + self.eps) + self.biases
    
class Dense_v2(tf.Module):
    def __init__(self, in_features:int, out_features:int, weights:tf.Tensor, bias:tf.Tensor=None, name=None):
        super().__init__(name=name)
        self.in_features = in_features
        self.out_features = out_features
        self.w = tf.reshape(weights, (in_features, out_features))
        if bias is None:
            self.b = tf.zeros([1, out_features])
        else:
            self.b = tf.reshape(bias, (1, out_features))
    # @tf.function
    def __call__(self, x:tf.Tensor):
        out = tf.matmul(x, self.w)
        return out + self.b

In [3]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.20f}".format(x)})
torch.set_printoptions(precision=20)

In [153]:
torch.set_default_device("cuda")
dtype = torch.float16 # test with float maybe
eps = 1e-5
# eps = torch.tensor(eps, dtype=torch.float16)
og_weights = torch.randn([4], dtype=torch.float)
og_bias = torch.randn([4], dtype=torch.float)
og_input = torch.randn([4], dtype=torch.float)
tf_weights = tf.constant(og_weights.cpu().numpy())
tf_bias = tf.constant(og_bias.cpu().numpy())
tf_input = tf.constant(og_input.cpu().numpy())
print(og_weights, tf_weights)

tensor([ 0.68212890625000000000, -0.33520507812500000000,
        -0.66308593750000000000,  0.19763183593750000000], device='cuda:0',
       dtype=torch.float16) tf.Tensor(
[0.68212890625000000000 -0.33520507812500000000 -0.66308593750000000000
 0.19763183593750000000], shape=(4,), dtype=float16)


In [160]:
torch_layer = torch.nn.LayerNorm(4, eps=eps) # if the 
torch_layer.weight = torch.nn.Parameter(og_weights)
torch_layer.bias = torch.nn.Parameter(og_bias)

my_layer = LayerNorm(tf_weights, tf_bias, eps=eps)

# tf.keras.backend.set_floatx('float64')
# gamma = tf.constant_initializer(tf_weights.numpy())
# beta = tf.constant_initializer(tf_bias.numpy())
# keras_layer = tf.keras.layers.LayerNormalization(axis=-1, epsilon=eps, gamma_initializer=gamma, beta_initializer=beta)
# print(keras_layer(tf_input))


print(torch_layer(og_input))
# print(torch.nn.functional.layer_norm(og_input, [4], og_weights, og_bias, eps))
print(my_layer(tf_input))



# dont forget to run as keras
# the torch LN might be internally casting as float32

tensor([-0.66259765625000000000,  1.55371093750000000000,
        -1.50390625000000000000,  0.98095703125000000000], device='cuda:0',
       dtype=torch.float16, grad_fn=<NativeLayerNormBackward0>)
tf.Tensor(
[-0.66406250000000000000 1.55468750000000000000 -1.50390625000000000000
 0.98046875000000000000], shape=(4,), dtype=float16)


In [ ]:
# testing linear layers
weight = torch.randn([4,4], dtype=torch.float16)
pt_linear = torch.nn.Linear(4, 4)
pt_linear.weight= torch.nn.Parameter(weight.numpy())
pt_linear.weight = torch.nn.Parameter(og_bias.numpy())
print(og_bias, og_bias.numpy())

pt_linear(og_input)

